In [13]:
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization
from tensorflow.keras.callbacks import LambdaCallback
import tensorflow as tf
import os

## Load train and test dataset

In [14]:
def load_dataset():
     # load dataset
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    
     # one hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

# load_dataset()

## Scale pixels

In [15]:
def prep_pixels(train, test):
 # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
 # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
 # return normalized images
    return train_norm, test_norm


## CNN model

In [17]:
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
     # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## Plot diagnostic learning curves

In [18]:
def summarize_diagnostics(history):
    
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()


## Run the test harness for evaluating a model

In [24]:
def run_test_harness():
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    # define model
    model = define_model()
    # fit model
    history = model.fit(trainX, trainY, epochs=30, batch_size=64, verbose=1, validation_data=(testX, testY))
    # save model
    model.save('final_model.h5')

    # Print the loss and accuracy for each epoch
    for i, loss in enumerate(history.history['loss']):
        print(f"Epoch {i+1}: loss = {loss:.4f}, accuracy = {history.history['accuracy'][i]:.4f}")

    _, acc = model.evaluate(testX, testY, verbose=0)
    print('> %.3f' % (acc * 100.0))

# entry point, run the test harness
run_test_harness()


Epoch 1/30
782/782 [==============================] - 97s 123ms/step - loss: 1.7198 - accuracy: 0.3806 - val_loss: 1.4595 - val_accuracy: 0.4753
Epoch 2/30
782/782 [==============================] - 99s 126ms/step - loss: 1.3696 - accuracy: 0.5070 - val_loss: 1.3030 - val_accuracy: 0.5237
Epoch 3/30
782/782 [==============================] - 96s 123ms/step - loss: 1.1962 - accuracy: 0.5757 - val_loss: 1.1796 - val_accuracy: 0.5836
Epoch 4/30
782/782 [==============================] - 96s 122ms/step - loss: 1.0636 - accuracy: 0.6254 - val_loss: 1.0406 - val_accuracy: 0.6315
Epoch 5/30
782/782 [==============================] - 101s 129ms/step - loss: 0.9683 - accuracy: 0.6613 - val_loss: 1.0575 - val_accuracy: 0.6268
Epoch 6/30
782/782 [==============================] - 105s 135ms/step - loss: 0.8762 - accuracy: 0.6936 - val_loss: 0.9045 - val_accuracy: 0.6840
Epoch 7/30
782/782 [==============================] - 100s 128ms/step - loss: 0.8080 - accuracy: 0.7208 - val_loss: 0.9096 - val

##  Load and prepare the image


In [29]:
def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(32, 32))
    img = img_to_array(img)

    # reshape into a single sample with 3 channels
    img = img.reshape(1, 32, 32, 3)
 
    # prepare pixel data
    img = img.astype('float32')
    img = img / 255.0
    return img

